# **Deep Learning with PyTorch** -  Introductory Lab

## **Part 2:** Image Operations with PyTorch

* In this part we will use PyTorch to operate on images.
* This notebook is designed to be run on Colab.
* But it can easily be modified to be run locally.

### Setup
* First we need to setup some things.
* Add the image data to you google drive by following these steps:
    1. Click on [this Google drive folder](https://drive.google.com/drive/folders/1M_5MmGsHMxbTraagutaw2T3Ii537qDrr?usp=sharing)
    2. Add a shortcut of that folder to you drive (in google drive, right-click on the filder and create shortcut)
    3. Run the code in the cell below and follow the instructions to mount your drive to Colab.
* If you instead want to run things locally on your computer, just download the data in the drive folder, and set the `dataset_path` to the location on your computer.

In [ ]:
# Connect your google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Set up path to your dataset
# Use this path if you have copied the data to your Google drive
# If you put the data somewhere else, you need to modify this
dataset_path = '/content/gdrive/My Drive/DIV2K_train_small/'

In [ ]:
# Import libraries we will need
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

### Reading and showing and image
* We will first try to read an image

In [ ]:
# We will use PIL to read an image
im_pil = Image.open(dataset_path + '0701x4.png')

# We first convert the image to a numpy array
im_np = np.array(im_pil)

# Numpy is the standard matrix library in Python. 
# But PyTorch effectively replaces it together with the functionality needed for deep learning.
# Still, we will enounter Numpy arrays in intermediate stages.
# Similar to PyTorch, we can check the shape of the tensor.
im_np.shape

In [ ]:
# Now lets display the image. We can do this with matplotlib
plt.imshow(im_np)   # Plot image
plt.axis('off')     # Just turns of the axis
plt.show()          # Finally show it

In [ ]:
# We can easily convert the image from numpy to PyTorch
im_torch = torch.from_numpy(im_np)

# Check shape
print(im_torch.shape)

# By default, PyTorch uses the data order C x H x W for images.
# So we should move the RGB channel dimension to the first dimension
im_torch = im_torch.permute(2, 0, 1)

print(im_torch.shape)

In [ ]:
# Lets check some details about the image
print('Type:', im_torch.dtype)
print('Min:', im_torch.min().item())
print('Max:', im_torch.max().item())

# Note that the image is a Byte tensor (uint8)
# For most image operations it is better to first convert it to floating point with values between 0 and 1

im = im_torch.float() / 255

print()
print('Type:', im.dtype)
print('Min:', im.min().item())
print('Max:', im.max().item())

* Lets write some convenient functions that performs reading and conversion of images in one step.
* We also write a function for showing a torch image.

In [ ]:
# Read an image with the given name and convert it to torch
def imread(image_file):
    im_pil = Image.open(dataset_path + image_file)
    im_np = np.array(im_pil, copy=False)
    im_torch = torch.from_numpy(im_np).permute(2, 0, 1)
    return im_torch.float()/255

# Show a PyTorch image tensor
def imshow(im, normalize=False):
    # Fit the image to the [0, 1] range if normalize is True
    if normalize:
        im = (im - im.min()) / (im.max() - im.min())

    # Remove redundant dimensions 
    im = im.squeeze()    # Mini excersize: check in the documentation what this function does

    is_color = (im.dim() == 3)

    # If there is a color channel dimension, move it to the end
    if is_color:
        im = im.permute(1, 2, 0)

    im_np = im.numpy().clip(0,1)    # Convert to numpy and ensure the values in the range [0, 1]
    if is_color:
        plt.imshow(im_np)
    else:
        plt.imshow(im_np, cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
# Lets try these functions
im = imread('0705x4.png')

imshow(im)

# Also check the type
print('Type:', im.dtype)

### Simple image operations

* Lets start by using pytorch to perform some simple operations on images.
* You can try around with the things you learned in the previous part of the lab.

In [ ]:
im = imread('0705x4.png')

# Lets start with a grayscale image.
# Color image can easily be converted to grayscale by simply averaging the color channels.

im_gray = im.mean(dim=0)

imshow(im_gray)

In [ ]:
# Try inverting the intensity
im_inv = 1 - im_gray
imshow(im_inv)

In [ ]:
# Map intensities
imshow(torch.sqrt(im_gray))
imshow(im_gray**2)

In [ ]:
# Crop the image
imshow(im_gray[:200, 130:310])

In [ ]:
# Flip the image along different axies
imshow(im_gray.flip(dims=(1,)))
imshow(im_gray.flip(dims=(0,)))

In [ ]:
# Mean of original image and its reflection
im2 = im_gray/2 + im_gray.flip(dims=(1,))/2
imshow(im2)

In [ ]:
# Note that this moves pixel values outside the range [0, 1]
imshow(im_gray.exp())

# You can visualize it better by setting the normalize flag
imshow(im_gray.exp(), normalize=True)

In [ ]:
# We can try swapping the order of the color channels

im_bgr = im[torch.LongTensor([2, 1, 0]), ...]
imshow(im_bgr)

im_rbg = im[torch.LongTensor([0, 2, 1]), ...]
imshow(im_rbg)

im_grb = im[torch.LongTensor([1, 0, 2]), ...]
imshow(im_grb)

### 💡 **Exercise**
* Create and show an image where only the green channel is reflected (i.e. flipped left-to-right) while the other channels are unchanged.

In [ ]:
# Implement your solution here


### 💡 **Exercise**
* Load three different images and construct a new image by taking one of the color channels from each image (ie, the red channel from the first image, the green from the second and the blue from the third).
* **Tip:** Check the `torch.stack` and `torch.cat` functions from the previous part.
* If the images have different sizes, extract a fix-sized crop as shown earlier.

In [ ]:
# Implement your solution here


### The convolution operation
* Now, lets take a look at the convolution operation.


In [ ]:
# First we need to import the nn.functional package in pytorch, which contains additional tensor operations.
import torch.nn.functional as F

In [ ]:
# Lets load our image again
im = imread('0705x4.png')
im_gray = im.mean(dim=0)

imshow(im_gray)

* The 2d convolution is performed using the `conv2d` function ([see doc](https://pytorch.org/docs/stable/nn.functional.html#conv2d))
* It takes two main inputs, namely the input tensor and the weight (i.e. filter)
* The input needs to be a 4-dimensional tensor with shape `B x C x H x W` where:
    - `B` is the 'batch size', i.e. the number of images. We currently only want to convolve one image, so `B=1` for now.
    - `C` is the number of channes (1 for grayscale and 3 for color).
    - `H x W` are the spatial dimensions.
* The weight need to be 4-dimensional with shape `D x C x kW x kH` where:
    - `D` is the number of output channels.
    - `kH x kW` is the kernel size (ie spatial size of the filter).

In [ ]:
# Lets first create an averaging filter

ksz = 9     # Size of the kernel

# We want grayscale input and output, so C = D = 1
weight = torch.ones(1, 1, ksz, ksz) / ksz**2

# Then need to resize the image to 4d
im_gray_4d = im_gray.view(1, 1, im_gray.shape[0], im_gray.shape[1])

# Now we can apply the convolution
im_out = F.conv2d(im_gray_4d, weight)

imshow(im_out)

* We see that the result is blurry.
* Try varying the kernel size `ksz` and see what happens.

### 💡 **Exercise**
* See how well you can remove noise with this convolution filter.
* You can add Gaussian noise to the image first using `torch.randn` (see part1 of the lab).

In [ ]:
# Implement your denoising solution here


### 💡 **Exercise**
* Compute x and y derivatives of a grayscale image using the filters shown in the lecture.

In [ ]:
# Implement your solution here

deriv_x_filter = None

deriv_y_filter = None

# ...


### In the next part we will write a deep learning example ...